In [ ]:
!pip install -i https://test.pypi.org/simple/ adawat

In [ ]:
from adawat.nlp import Corpus

#corpus_path='/home/rafid/Downloads/wiki-sents/wiki-sents.txt'
#corpus_path='../input/wikipedia-sentences/wikisent2.txt'\
corpus_path='./text8'

def to_lowercase(line: str) -> str:
    return line.lower()

#MAX_LINES = 100_000
#MAX_VOCAB = 1_000
#corpus = Corpus(corpus_path, MAX_LINES, MAX_VOCAB, preprocessor=to_lowercase)

MAX_VOCAB = 30_000
corpus = Corpus(corpus_path, max_vocab=MAX_VOCAB)


len(corpus.tokens), corpus.vocab_size

In [ ]:
from typing import Tuple

CONTEXT_RADIUS = 5

def build_training_data(corpus: Corpus, context_radius=2) -> Tuple:
    features = []
    targets = []
    for tokens in corpus.tokens_per_line:
        for i in range(context_radius, len(tokens) - context_radius):
            features.append(
                tokens[i - context_radius:i] +
                tokens[i + 1: i + context_radius + 1]
            )
            targets.append(tokens[i])
    return features, targets

features, targets = build_training_data(corpus, CONTEXT_RADIUS)
list(zip(features[0:3], targets[0:3]))


In [ ]:
import torch
from torch import nn
from torch.nn import functional as F


EMBEDDING_DIM = 300

class CBOW(nn.Module):
    def __init__(self, vocab_size, context_radius, embedding_dim):
        super(CBOW, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(2 * context_radius * embedding_dim, 512)
        self.linear2 = nn.Linear(512, vocab_size)

    def forward(self, input):
        output1 = self.embedding(input)
        output1 = torch.flatten(output1, 1)
        output2 = F.relu(self.linear1(output1))
        output3 = self.linear2(output2)

        return F.log_softmax(output3, dim=1)


def model_func(vocab_size, context_radius, embedding_dim):
    """
     A function for defining instances of the model for use with the ModelTrainer class
    """

    def model():
        return CBOW(vocab_size, context_radius, embedding_dim)

    return model


def test_model():
    model_fn = model_func(corpus.vocab_size, CONTEXT_RADIUS, EMBEDDING_DIM)
    model = model_fn()
    output = model(torch.tensor(
        [list(range(0, 2*CONTEXT_RADIUS))] * 10
    ))
    print(output)
    print(output.shape)


test_model()


In [ ]:
if torch.cuda.is_available():
    print("CUDA is available! Using it.")
    device = torch.device("cuda:0")
else:
    print("CUDA is not available :'( Using CPU.")
    device = torch.device("cpu")


In [ ]:
# Python imports
from itertools import islice

# PyTorch impots
from torch.utils.data import DataLoader

# 3rd party imports
from adawat.data import ListDataset
from adawat.transforms import Compose, WordToIndex, WordsToIndices, \
    WordToOneHot, ToPyTorchTensor


BATCH_SIZE = 100

dataset = ListDataset(
    features, targets,
    # Features transformation
    Compose(
        WordsToIndices(corpus.word2idx),
        ToPyTorchTensor(dtype=torch.long, device=device)
    ),
    # Targets transformation
    Compose(
        #WordToOneHot(corpus.word2idx, corpus.vocab_size),
        WordToIndex(corpus.word2idx),
        ToPyTorchTensor(dtype=torch.long, device=device)
    )
)
data_loader = DataLoader(dataset, BATCH_SIZE)

In [ ]:
# Configure logging
def configure_logging():
    import logging
    root = logging.getLogger()
    root.setLevel(logging.DEBUG)

    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)

    root.handlers = [ch]
configure_logging()


In [ ]:
# Python imports
import random

import numpy as np

# PyTorch imports
from torch import optim

# Other imports
from adawat.models import ModelTrainer


def closest_vector(vector, vectors):
    vectors = np.asarray(vectors)
    dist_2 = np.sum((vectors - vector)**2, axis=1)
    dist_2_sorted = sorted(zip(dist_2, range(len(dist_2))))
    return dist_2_sorted[1][1]


def get_word_embedding(embedding_matrix, word, word2idx):
    idx = word2idx(word)
    return embedding_matrix(torch.tensor(idx, device=device))


def train():
    model_fn = model_func(corpus.vocab_size, CONTEXT_RADIUS, EMBEDDING_DIM)
    model_trainer = ModelTrainer(model_fn, nn.NLLLoss,
                                 lambda parameters: optim.SGD(parameters, 0.001))
    model, losses = model_trainer.train(data_loader, 3, device)
    embedding = model.embedding
    embedding_matrix = model.embedding.weight
    
    for word in random.choices(corpus.vocab, k=100):
        word_vector = get_word_embedding(embedding, word, corpus.word2idx)
        closest_idx = closest_vector(word_vector.detach().cpu().numpy(),
                                 embedding_matrix.detach().cpu().numpy())
        print(f"The closest word to {word} is {corpus.idx2word(closest_idx)}")


train()
